In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
df = pd.read_csv('../week03/data/train.csv')

In [4]:
df = pd.read_csv('../week03/data/train.csv', parse_dates=[0])

In [5]:
df['day']= df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month

In [6]:
df.loc[df['weather']==4, 'weather'] = 3

In [7]:
from sklearn.model_selection import (train_test_split, cross_val_score, GridSearchCV)

In [8]:
y = df['count'] # this is the dv
X = df.drop(['count', 'registered', 'casual', 'atemp'],  axis =1) #prevents data leaking, also atemp isn't useful

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=14)

In [10]:
num_features = ['temp', 'humidity', 'windspeed',]
cat_features = ['season', 'holiday', 'workingday', 'weather', 'hour', 'day','month',]

features = num_features + cat_features

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  
# turn categorical values into binary columns, ordinal, bins, and scales
from sklearn.preprocessing import (OneHotEncoder, OrdinalEncoder, KBinsDiscretizer, MinMaxScaler, StandardScaler, PolynomialFeatures)
#column transform
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import (LinearRegression, Lasso, Ridge, ElasticNet)

In [12]:
num_trans= Pipeline(
   
   [ ('polynomial', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    
    
   ]    
)

In [13]:
cat_trans = Pipeline(
   [
        ('ohe', OneHotEncoder(drop="first")),
        ('polynomial', PolynomialFeatures()),
    ]
)

In [14]:
ct = ColumnTransformer(
    [
        ('num', num_trans, num_features),
        ('cat', cat_trans, cat_features),
    ]
)

In [15]:
estimator = Pipeline(
    [
    ('ct', ct), #this should do all of it?
    ('lasso', Lasso(alpha = 0.1))#this is for the goddamn lasso
    ]
)

In [16]:
estimator.fit(X_train, y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('polynomial',
                                                                   PolynomialFeatures()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'humidity',
                                                   'windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures())]),
                                                  ['season', 'holiday',
                                                   'workingday', 'weather',
                                                   'hour', 'day', 'month'])])),
                ('lasso', Lasso(alpha=0.1))])

from below here I am really imitating, at least above I am somewhat familiar with, though I admit Carlos actually has a very clean pipeline process, it forces you to 
categorize your variables first into a list, then develop 2 pipelines one for each and then push it all into one big one. I wonder how it works when there are more than 2, one that needs more attention?

In [17]:
estimator.score(X_test,y_test)

0.7979877783466902

In [18]:
#tf am I even looking for 
estimator.get_params()

{'memory': None,
 'steps': [('ct', ColumnTransformer(transformers=[('num',
                                    Pipeline(steps=[('polynomial',
                                                     PolynomialFeatures()),
                                                    ('scaler', StandardScaler())]),
                                    ['temp', 'humidity', 'windspeed']),
                                   ('cat',
                                    Pipeline(steps=[('ohe',
                                                     OneHotEncoder(drop='first')),
                                                    ('polynomial',
                                                     PolynomialFeatures())]),
                                    ['season', 'holiday', 'workingday', 'weather',
                                     'hour', 'day', 'month'])])),
  ('lasso', Lasso(alpha=0.1))],
 'verbose': False,
 'ct': ColumnTransformer(transformers=[('num',
                                  Pipeline(steps

In [19]:
#This dictionary I want to make a note on every line so its very clear what I am doing
param_grid = {
    'ct__cat__polynomial__degree': [2,3],
    'ct__cat__polynomial__interaction_only': [False,True],
    'lasso__alpha': [100.,10.,1.,0.1,0.01],
    'lasso__max_iter': [5_000, 10_000,20_000]
}

In [20]:
#so this calls the estimator pipeline, the param grid, scores for r2, 
gscv = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='r2',
    cv=4, 
    n_jobs=-1,
    verbose=1
)

In [21]:
# grid-search cross-validation
#this took like a half hour at full processesing on my mac, so don't run past here unless you have decent processing power
#on my mac it seems to take roughly 5 minutes per additional fold
gscv.fit(X_train,y_train)
print(f"time taken: {round(gscv.refit_time_,2)} sec")


Fitting 4 folds for each of 60 candidates, totalling 240 fits
time taken: 72.09 sec


In [22]:
gscv.best_params_

{'ct__cat__polynomial__degree': 3,
 'ct__cat__polynomial__interaction_only': False,
 'lasso__alpha': 0.1,
 'lasso__max_iter': 5000}

In [23]:
round(gscv.best_score_,6)

0.815183

In [24]:
gscv.best_estimator_

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('polynomial',
                                                                   PolynomialFeatures()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'humidity',
                                                   'windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(degree=3))]),
                                                  ['season', 'holiday',
                                                   'workingday', 'weather',
                                                   'hour', 'day', 'month'])])),
                ('lasso', Lasso(alpha=0.1, max_iter=5000))])

In [25]:
best_model= gscv.best_estimator_

In [26]:
best_model.fit(X_train,y_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('polynomial',
                                                                   PolynomialFeatures()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'humidity',
                                                   'windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(degree=3))]),
                                                  ['season', 'holiday',
                                                   'workingday', 'weather',
                                                   'hour', 'day', 'month'])])),
                ('lasso', Lasso(alpha=0.1, max_iter=5000))])

In [27]:
# training score
training_score = best_model.score(X_train,y_train)

# test score
test_score = best_model.score(X_test,y_test)

print(training_score)
print(test_score)

0.8367084337493538
0.8048522206941996


In [44]:
numerical_transformer = Pipeline(
    steps=[
        ('expand_features', PolynomialFeatures())
    ]
)

random forest below, this might also have a relatively long thing

In [46]:
from sklearn.ensemble import RandomForestRegressor

In [47]:
estimator = Pipeline(
    steps=[
        ('ct', ct), # preprocessing step
        ('rf', RandomForestRegressor()) # random forest regression
    ]
)

In [48]:
param_grid = {
    'rf__n_estimators': [50,100,200,300,500],
    'rf__max_depth': [5,10,20,50,100,None],
    'rf__min_samples_split': [2, 5, 10]
}

In [49]:
gscv = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='r2',
    cv=4, 
    n_jobs=-1,
    verbose=1
)

In [50]:
gscv.fit(X_train,y_train)

print(f"time taken: {round(gscv.refit_time_,2)} sec")

Fitting 4 folds for each of 90 candidates, totalling 360 fits


KeyboardInterrupt: 

In [ ]:
#god that last one was well over an hour time wise
round(gscv.best_score_,6)